# PyTorch Assignment Answer --- MNIST Classifier

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F

In [ ]:
# Setting hyper parameters

batch_size = 32
learning_rate = 1e-2
num_epoches = 50

In [ ]:
# Downloading dataset

train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Construct your model to classify image. Your model should 
# contain one hidden layer. And both layeres is FC (fully connected). 

# Remember adding a non-linear function (usually Relu) between two layers. (Think about why)

class Neuralnetwork(nn.Module):
    # Your model should be given 3 parameters: input_dimension, hidden_dimension and output_dimension
    
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(in_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, out_dim)
        
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        
        return x

input_dimension = 28 * 28
hidden_dimension = 300
output_dimension = 10

model = Neuralnetwork(input_dimension, hidden_dimension, output_dimension)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Train your model

for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        # getting training data
        img, label = data
        # reshape image from 28 * 28 tensors into 784 dimension vectors
        img = img.view(img.size(0), -1)
        img = Variable(img)
        # label is an integer representing the number respect to the pictures
        label = Variable(label)
        
        out = model(img)
        loss = criterion(out, label)
        
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print training loss and accuracy on training set
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
    epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
    train_dataset))))

In [ ]:
# Test your model on test set. 

model.eval()
eval_loss = 0.
eval_acc = 0.
for data in test_loader:
    img, label = data
    img = img.view(img.size(0), -1)
    img = Variable(img, volatile=True)
    label = Variable(label, volatile=True)
    
    out = model(img)
    loss = criterion(out, label)
    
    eval_loss += loss.data[0] * label.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == label).sum()
    eval_acc += num_correct.data[0]

# print test loss and accuracy on test set
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
    test_dataset)), eval_acc / (len(test_dataset))))